In [17]:
from pymilvus import Milvus, DataType

from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility


## sample milvus


In [18]:
book_id = FieldSchema(
  name="book_id", 
  dtype=DataType.INT64, 
  is_primary=True, 
)
word_count = FieldSchema(
  name="word_count", 
  dtype=DataType.INT64,  
)
book_intro = FieldSchema(
  name="book_intro", 
  dtype=DataType.FLOAT_VECTOR, 
  dim=2
)
schema = CollectionSchema(
  fields=[book_id, word_count, book_intro], 
  description="Test book search"
)
collection_name = "book"

In [20]:
connections.connect()


In [21]:
from pymilvus import Collection
collection = Collection(
    name=collection_name, 
    schema=schema, 
    using='default', 
    shards_num=2,
    consistency_level="Strong"
    )

In [22]:
from pymilvus import utility
utility.has_collection("book")

True

In [23]:
from pymilvus import Collection
collection = Collection("book")

In [ ]:
collection

<Collection>:
-------------
<name>: book
<partitions>: [{"name": "_default", "collection_name": "book", "description": ""}]
<description>: Test book search
<schema>: {
  auto_id: False
  description: Test book search
  fields: [{
    name: book_id
    description: 
    type: 5
    is_primary: True
    auto_id: False
  }, {
    name: word_count
    description: 
    type: 5
  }, {
    name: book_intro
    description: 
    type: 101
    params: {'dim': 2}
  }]
}

In [ ]:
collection

<Collection>:
-------------
<name>: book
<partitions>: [{"name": "_default", "collection_name": "book", "description": ""}]
<description>: Test book search
<schema>: {
  auto_id: False
  description: Test book search
  fields: [{
    name: book_id
    description: 
    type: 5
    is_primary: True
    auto_id: False
  }, {
    name: word_count
    description: 
    type: 5
  }, {
    name: book_intro
    description: 
    type: 101
    params: {'dim': 2}
  }]
}

In [ ]:
from pymilvus import utility
utility.list_collections()

['question_answering', 'book']

In [4]:
import random
data = [
  [i for i in range(2000)],
  [i for i in range(10000, 12000)],
  [[random.random() for _ in range(2)] for _ in range(2000)],
]

In [5]:
data[0][0], data[1][0], data[2][0]

(0, 10000, [0.5357308618286432, 0.9081746943842883])

In [ ]:
from pymilvus import Collection
collection = Collection("book")      # Get an existing collection.
mr = collection.insert(data)

In [ ]:
mr

(insert count: 2000, delete count: 0, upsert count: 0, timestamp: 432813323722686467)

In [ ]:
expr = "book_id in [0,1]"
collection.delete(expr)



(insert count: 0, delete count: 2, upsert count: 0, timestamp: 432813348180983811)

In [ ]:
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}



In [ ]:
collection.load()

In [ ]:
results = collection.search(
	data=[[0.1, 0.2]], 
	anns_field="book_intro", 
	param=search_params, 
	limit=10, 
	expr=None,
	consistency_level="Strong"
)

In [ ]:
results[0].ids

[752, 1295, 1278, 413, 639, 293, 1429, 1916, 77, 909]

In [ ]:
results[0].distances



[1.5257634004228748e-05, 0.00041668565245345235, 0.0004344903863966465, 0.00044145507854409516, 0.0005489669856615365, 0.0008601932204328477, 0.0012088499497622252, 0.0013795846607536077, 0.0014143632724881172, 0.001415766542777419]

In [ ]:
search_param = {
  "data": [[0.1, 0.2]],
  "anns_field": "book_intro",
  "param": {"metric_type": "L2", "params": {"nprobe": 10}},
  "limit": 2,
  "expr": "word_count <= 11000",
}
res = collection.search(**search_param)

In [ ]:
res = collection.query(
  expr = "book_id in [2,4,6,8]", 
  output_fields = ["book_id", "book_intro"],
  consistency_level="Strong"
)

In [ ]:
for r in res:
    print(r)

{'book_id': 2, 'book_intro': [0.327368, 0.01204]}
{'book_id': 4, 'book_intro': [0.076528, 0.280461]}
{'book_id': 6, 'book_intro': [0.89448, 0.456958]}
{'book_id': 8, 'book_intro': [0.504008, 0.190562]}


In [6]:
fp = r"train_tam_qa.csv"

In [8]:
!pip install tqdm

  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)


In [9]:
import pandas as pd
from tqdm.auto import tqdm
import numpy

/home/cts-16/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
!pip install datasets

     |████████████████████████████████| 325 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 9.6 MB/s  eta 0:00:01
     |████████████████████████████████| 1.3 MB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 12.0 MB/s eta 0:00:01
  Using cached huggingface_hub-0.5.1-py3-none-any.whl (77 kB)
     |████████████████████████████████| 128 kB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 212 kB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 26.7 MB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 308 kB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 11.2 MB/s eta 0:00:01
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
     |████████████████████████████████| 121 kB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 9.5 MB/s  eta 0:00:01
  Using cached typing_extensions-4.2.0-py3-none-any.whl (24 kB)
  Using cached urllib3-1.26.9-py2.py3-none-any.whl 

In [15]:
from datasets import Dataset

In [10]:
with open(fp, "r") as read_file:
  qa = pd.read_json(read_file)

In [20]:
dataset = Dataset.from_pandas(qa)
dataset[0]

{'context': 'ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின்வரும் 206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட்டால் 208) எண்ணிக்கையான எலும்புகளைக் கொண்டிருக்கும். இந்த எண்ணிக்கை உடற்கூட்டியல் வேறுபாடுகளைப் பொறுத்து மாறுபடக்கூடும். எடுத்துக்காட்டாக, மிகக் குறைந்த எண்ணிக்கையான மனிதர்களில், ஒரு மேலதிக விலா எலும்பு (கழுத்துவில்) அல்லது ஒரு மேலதிகமான கீழ் முதுகெலும்பு காணப்படுவதுண்டு; இணைந்த சில எலும்புகளைத் தனி எலும்பாகக் கருதாவிடின், ஐந்து இணைந்த திருவெலும்பு; மூன்று (3 - 5) குயிலலகு எலும்புகள் சேர்ந்து 26 எண்ணிக்கையிலான முதுகெலும்புகள் 33 ஆகக் கருதப்படலாம்.\nமனித மண்டையோட்டில் 22 எலும்புகள் (காதுச் சிற்றென்புகளைத் தவிர) உள்ளன; இவை எட்டு மண்டையறை (cranium) எலும்புகளாகவும் 14 முக எலும்புகளாகவும் (facial bones) பிரிக்கப்பட்டுள்ளன. (தடித்த எண்கள் அருகிலுள்ள படத்தில் காணும் எண்களைக் குறிக்கின்றன.)\n\n\n\nமண்டையறை எலும்புகள் (8)\n 1 நுதலெலும்பு (frontal bone) \n 2 சுவரெலும்பு (parietal bone) (2)\n 3 கடைநுதலெலும்பு (temporal bone) (2)\n 4 பிடர் எலும்பு (occipital bone)\n ஆப்புரு எலும்பு (sphen

In [11]:
qa

,context,question,answer_text,answer_start
0,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53
1,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358
2,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0
3,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68
4,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585
...,...,...,...,...
363,இந்திய ரூபாய் நாணயங்கள் (Coins of the Indian r...,இந்திய நாணய சட்டம் எந்த ஆண்டு இயற்றப்பட்டது?,1955 ஆம் ஆண்டு செப்டம்பர்,5154
364,"ஜெர்மனி (Germany, [ˈdʒɜːmənɪ]), அல்லது ஜெர்மன்...",ஜெர்மனியில் மிகப்பெரிய மதம் எது?,கிறித்தவத்தை,30737
365,"கெய்ரோ (Cairo, அரபு மொழியில்: القاهرة - அல்-கா...",கெய்ரோ நகரத்தின் பரப்பளவு என்ன?,453 square kilometers,1507
366,வாழைப்பழம் (banana) என்பது தாவரவியலில் சதைப்பற...,மிகவும் அரிதான வாழைப்பழம் என்றால் என்ன?,சிங்கன்,7688


In [21]:
qa.to_json("tamil_qa.json", force_ascii=False)

In [23]:
!pip install transformers

  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
  Using cached sacremoses-0.0.49-py3-none-any.whl (895 kB)
  Using cached regex-2022.4.24-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (764 kB)
  Using cached tokenizers-0.12.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)


# mysql python

In [1]:
!pip install pymysql

In [2]:
import pymysql

In [4]:
conn = pymysql.connect(host='localhost', user='aswin', port=3306, password='Mysql@123', database='ODQA',local_infile=True)
cursor = conn.cursor()

In [5]:
TABLE_NAME = 'context'
#Deleting previouslny stored table for clean run
drop_table = "DROP TABLE IF EXISTS " + TABLE_NAME + ";"
cursor.execute(drop_table)

try:
    sql = "CREATE TABLE if not exists " + TABLE_NAME + " (id INT, context TEXT, PRIMARY KEY (ID));"
    sql = f"""
                CREATE TABLE if not exists {TABLE_NAME} (
                    id int(10) NOT NULL AUTO_INCREMENT,
                    context TEXT COLLATE utf8_bin NOT NULL, 
                    PRIMARY KEY (id)
                ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin
                AUTO_INCREMENT=1 ;"""
    cursor.execute(sql)
    print("create MySQL table successfully!")
except Exception as e:
    print("can't create a MySQL table: ", e)

create MySQL table successfully!


In [75]:
rows = cursor.fetchall()
for row in rows:
    # print(f'{row[0]} {row[1]} {row[2]}')
    print(row)


In [76]:
sql_query = "select * from context;"
cursor.execute(sql_query)


0

In [60]:
rows = cursor.fetchall()

In [61]:
for row in rows:
    print(rows)

(('1', 'sample'), ('1', 'ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின்வரும் 206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட்டால் 208) எண்ணிக்கையான எலும்புகளைக் கொண்டிருக்கும். இந்த எண்ணிக்கை உடற்கூட்டியல் வேறுபாடுகளைப் பொறுத்து மாறுபடக்கூடும். எடுத்துக்காட்டாக, மிகக் குறைந்த எண்ணிக்கையான மனிதர்களில், ஒரு மேலதிக விலா எலும்பு (கழுத்துவில்) அல்லது ஒரு மேலதிகமான கீழ் முதுகெலும்பு காணப்படுவதுண்டு; இணைந்த சில எலும்புகளைத் தனி எலும்பாகக் கருதாவிடின், ஐந்து இணைந்த திருவெலும்பு; மூன்று (3 - 5) குயிலலகு எலும்புகள் சேர்ந்து 26 எண்ணிக்கையிலான முதுகெலும்புகள் 33 ஆகக் கருதப்படலாம்.\nமனித மண்டையோட்டில் 22 எலும்புகள் (காதுச் சிற்றென்புகளைத் தவிர) உள்ளன; இவை எட்டு மண்டையறை (cranium) எலும்புகளாகவும் 14 முக எலும்புகளாகவும் (facial bones) பிரிக்கப்பட்டுள்ளன. (தடித்த எண்கள் அருகிலுள்ள படத்தில் காணும் எண்களைக் குறிக்கின்றன.)\n\n\n\nமண்டையறை எலும்புகள் (8)\n 1 நுதலெலும்பு (frontal bone) \n 2 சுவரெலும்பு (parietal bone) (2)\n 3 கடைநுதலெலும்பு (temporal bone) (2)\n 4 பிடர் எலும்பு (occipital bone)\n ஆப்புரு எல

In [58]:
desc = cursor.description

In [59]:
desc

(('id', 252, None, 262140, 262140, 0, True),
 ('context', 252, None, 262140, 262140, 0, True))

In [34]:
import importlib
importlib.reload(odqa_mysql)
import odqa_mysql


odqa


In [35]:
odqa_mysql.create_table()

context table successfully!


In [38]:
q = "show tables"
q = "SELECT DATABASE();"
q = "select count(id) from context"
res = odqa_mysql.execute_query(q)

In [41]:
res[0][0]

0

In [42]:
cursor.execute(q)

1

In [43]:
res = cursor.fetchone()

In [44]:
res

(0,)

In [6]:
id1 = 1
context = "ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின்வரும் 206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட்டால் 208) எண்ணிக்கையான எலும்புகளைக் கொண்டிருக்கும். இந்த எண்ணிக்கை உடற்கூட்டியல் வேறுபாடுகளைப் பொறுத்து மாறுபடக்கூடும். எடுத்துக்காட்டாக, மிகக் குறைந்த எண்ணிக்கையான மனிதர்களில், ஒரு மேலதிக விலா எலும்பு (கழுத்துவில்) அல்லது ஒரு மேலதிகமான கீழ் முதுகெலும்பு காணப்படுவதுண்டு; இணைந்த சில எலும்புகளைத் தனி எலும்பாகக் கருதாவிடின், ஐந்து இணைந்த திருவெலும்பு; மூன்று (3 - 5) குயிலலகு எலும்புகள் சேர்ந்து 26 எண்ணிக்கையிலான முதுகெலும்புகள் 33 ஆகக் கருதப்படலாம்.\nமனித மண்டையோட்டில் 22 எலும்புகள் (காதுச் சிற்றென்புகளைத் தவிர) உள்ளன; இவை எட்டு மண்டையறை (cranium) எலும்புகளாகவும் 14 முக எலும்புகளாகவும் (facial bones) பிரிக்கப்பட்டுள்ளன. (தடித்த எண்கள் அருகிலுள்ள படத்தில் காணும் எண்களைக் குறிக்கின்றன.)\n\n\n\nமண்டையறை எலும்புகள் (8)\n 1 நுதலெலும்பு (frontal bone) \n 2 சுவரெலும்பு (parietal bone) (2)\n 3 கடைநுதலெலும்பு (temporal bone) (2)\n 4 பிடர் எலும்பு (occipital bone)\n ஆப்புரு எலும்பு (sphenoid bone)\n நெய்யரியெலும்பு (ethmoid bone)\nமுக எலும்புகள் (14)\n 7 கீழ்த்தாடை எலும்பு (mandible)\n 6 மேற்றாடை எலும்பு (maxilla) (2)\n அண்ணவெலும்பு (palatine bone) (2)\n 5 கன்ன எலும்பு (zygomatic bone) (2)\n 9 நாசி எலும்பு (nasal bone) (2)\n கண்ணீர் எலும்பு (lacrimal bone) (2)\n மூக்குச் சுவர் எலும்பு (vomer)\n கீழ் மூக்குத் தடுப்பெலும்பு (inferior nasal conchae) (2)\nநடுக்காதுகளில் (6):\n சம்மட்டியுரு (malleus)\n பட்டையுரு (incus) \n ஏந்தியுரு (stapes)\nதொண்டையில் (1):\n தொண்டை எலும்பு (நாவடி எலும்பு) (hyoid)\nதோள் பட்டையில் (4):\n 25. காறை எலும்பு (clavicle)\n 29. தோள் எலும்பு (scapula)\nமார்புக்கூட்டில் thorax(25): \n 10. மார்பெலும்பு (sternum) (1)\n மேலும் மூன்று என்புகளாகக் கருதப்படலாம்: பிடியுரு (manubrium), உடல் மார்பெலும்பு (body of sternum), வாள்வடிவ நீட்டம் (xiphoid process)\n 28. விலா எலும்புகள் (rib) (24)\n முதுகெலும்புத் தூண் (vertebral column) (33): \n 8. கழுத்து முள்ளெலும்புகள் (cervical vertebra) (7)\n மார்பு முள்ளெலும்புகள் (thoracic vertebra) (12)\n 14. நாரிமுள்ளெலும்புகள் (lumbar vertebra) (5)\n 16. திரிகம் (திருவெலும்பு) (sacrum)\n வால் எலும்பு (குயிலலகு) (coccyx)\nமேற்கைகளில் (arm) (1): \n 11. புய எலும்பு (மேல்கை எலும்பு) (humerus)\n 26. புய எலும்புப் புடைப்பு (மேல்கை எலும்புப் புடைப்பு) (condyles of humerus)\nமுன்கைகளில் (forearm) (4): \n 12. அரந்தி (ulna) (2)\n 13. ஆரை எலும்பு (radius) (2)\n 27. ஆரை எலும்புத் தலை (head of radius)\nகைகளில் (hand) (54): \n மணிக்கட்டுகள் (carpal):\n படகெலும்பு (scaphoid) (2)\n பிறைக்குழி எலும்பு (lunate) (2)\n முப்பட்டை எலும்புtriquetrum) (2)\n பட்டாணி எலும்பு (pisiform) (2)\n சரிவக எலும்பு (trapezium) (2)\n நாற்புறவுரு எலும்பு (trapezoid) (2)\n தலையுரு எலும்பு (capitate) (2)\n கொக்கி எலும்பு (hamate) (2)\n அங்கை முன்னெலும்புகள் (அனுமணிக்கட்டு எலும்புகள்) (metacarpal): (5 × 2)\n விரலெலும்புகள் (phalange): \n அண்மை விரலெலும்புகள் (proximal phalanges) (5 × 2)\n நடு விரலெலும்புகள் (Intermediate phalanges) (4 × 2)\n தொலை விரலெலும்புகள் (distal phalanges) (5 × 2)\nஇடுப்பு வளையம் (pelvis) (2):\n 15. இடுப்பெலும்பு (ilium) மற்றும் கீழ் இடுப்பெலும்பு (ischium)\n\n\nகால்கள் (leg) (8): \n 18. தொடையெலும்பு (femur) (2)\n 17. இடுப்பு மூட்டு (hip joint) (மூட்டு, எலும்பல்ல) \n 22. பெரிய தொடையெலும்புக் கொண்டை (greater trochanter of femur)\n 23. தொடையெலும்புப் புடைப்பு (condyles of femur)\n 19. சில்லெலும்பு (patella) (2)\n 20. கால் முன்னெலும்பு (கணைக்காலலுள்ளெலும்பு) (tibia) (2)\n 21. சிம்பு எலும்பு (கணைக்கால்வெளியெலும்பு) (fibula) (2)\nகாலடிகளில் (52): \n கணுக்காலெலும்புகள் (tarsal):\n குதிகால் (calcaneus) (2)\n முட்டி (talus) (2)\n படகுரு எலும்பு (navicular bone) (2)\n உள் ஆப்புவடிவ எலும்பு (2) \n இடை ஆப்புவடிவ எலும்பு (2)\n வெளி ஆப்புவடிவ எலும்பு (2)\n கனசதுர எலும்பு (cuboidal bone) (2)\n அனுகணுக்காலெலும்புகள் (metatarsal) (5 × 2)\n விரலெலும்புகள் (phalange):\n அண்மை விரலெலும்புகள் (proximal phalanges) (5 × 2)\n நடு விரலெலும்புகள் (intermediate phalanges) (4 × 2)\n தொலை விரலெலும்புகள் (distal phalanges) (5 × 2)\n குழந்தை எலும்புக்கூடு \nகுழந்தைகளின் எலும்புக்கூடுகளில் கீழ் வரும் எலும்புகள் மேலதிகமாக உள்ளன:\n மண்டையறை மற்றும் மண்டையோட்டு எலும்புகள் (21), இவை ஒன்றாகி மண்டையறையை உருவாக்குகின்றன.\n திரிக முள்ளெலும்புகள் (sacral vertebrae) (4 or 5), வளர்ந்தோரில் இவை ஒன்றாகி திரிகத்தை உருவாக்குகின்றன \n coccygeal vertebrae (3 to 5), வளர்ந்தோரில் இவை ஒன்றாகி வாலெலும்பை உருவாக்குகின்றன\n இடுப்பெலும்பு, கீழ் இடுப்பெலும்பு மற்றும் பொச்செலும்பு (pubis), என்பவை வளர்ந்தோரில் ஒன்றாகி இடுப்பெலும்பை உருவாக்குகின்றன\n\nபகுப்பு:மனித உடற்கூற்றியல்\nபகுப்பு:எலும்புகள்"
# context = "sample"

In [7]:
# q = ("INSERT INTO context values(%i, %s)" , (int(id1), context))
# q

In [8]:
# cursor.execute(f'INSERT INTO context values(%i, %s)' , (int(id1), context)) 
# cursor.execute(f'INSERT INTO context values(1, context)') 
sql = "INSERT INTO context (context) VALUES (%s)"
cursor.execute(sql, (context))

conn.commit()
print('new city inserted')

new city inserted


In [10]:
q = f"select context from context"
res = cursor.execute(q)

In [11]:
res = cursor.fetchall()

In [12]:
res

(('ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின்வரும் 206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட்டால் 208) எண்ணிக்கையான எலும்புகளைக் கொண்டிருக்கும். இந்த எண்ணிக்கை உடற்கூட்டியல் வேறுபாடுகளைப் பொறுத்து மாறுபடக்கூடும். எடுத்துக்காட்டாக, மிகக் குறைந்த எண்ணிக்கையான மனிதர்களில், ஒரு மேலதிக விலா எலும்பு (கழுத்துவில்) அல்லது ஒரு மேலதிகமான கீழ் முதுகெலும்பு காணப்படுவதுண்டு; இணைந்த சில எலும்புகளைத் தனி எலும்பாகக் கருதாவிடின், ஐந்து இணைந்த திருவெலும்பு; மூன்று (3 - 5) குயிலலகு எலும்புகள் சேர்ந்து 26 எண்ணிக்கையிலான முதுகெலும்புகள் 33 ஆகக் கருதப்படலாம்.\nமனித மண்டையோட்டில் 22 எலும்புகள் (காதுச் சிற்றென்புகளைத் தவிர) உள்ளன; இவை எட்டு மண்டையறை (cranium) எலும்புகளாகவும் 14 முக எலும்புகளாகவும் (facial bones) பிரிக்கப்பட்டுள்ளன. (தடித்த எண்கள் அருகிலுள்ள படத்தில் காணும் எண்களைக் குறிக்கின்றன.)\n\n\n\nமண்டையறை எலும்புகள் (8)\n 1 நுதலெலும்பு (frontal bone) \n 2 சுவரெலும்பு (parietal bone) (2)\n 3 கடைநுதலெலும்பு (temporal bone) (2)\n 4 பிடர் எலும்பு (occipital bone)\n ஆப்புரு எலும்பு (sphenoid bone)\

In [67]:
q = "truncate table context;"
cursor.execute(q)


KeyboardInterrupt: 

In [13]:
!pip install transformers

  Using cached sacremoses-0.0.49-py3-none-any.whl (895 kB)


In [16]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
